<a href="https://colab.research.google.com/github/QaziSaim/Fine-Tune-Projects/blob/main/Attention_Mechenisum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Luckly I already had saved my tokenizer now i am going to load it

In [23]:
import numpy as np
import pandas as pd
import tensorflow
import re
import string
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Layer

In [13]:
df = pd.read_csv('/content/eng_to_fra.csv')

In [14]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(df.eng_texts)
eng_sequence = eng_tokenizer.texts_to_sequences(df.eng_texts)

In [15]:
fra_tokenizer = Tokenizer()
fra_tokenizer.fit_on_texts(df.fra_texts)
fra_sequence = fra_tokenizer.texts_to_sequences(df.fra_texts)

In [16]:
eng_vocab_size = len(eng_tokenizer.word_index) +1
fra_vocab_size = len(fra_tokenizer.word_index) + 1


In [17]:
max_eng_len = max(len(x) for x in eng_sequence)
max_fra_len = max(len(x) for x in fra_sequence)

In [18]:
encoder_input = pad_sequences(eng_sequence, maxlen=max_eng_len, padding='post')
decoder_input = pad_sequences([s[:-1] for s in fra_sequence],maxlen=max_fra_len-1,padding='post')
decoder_target = pad_sequences([s[1:] for s in fra_sequence],maxlen=max_fra_len-1,padding='post')

In [19]:
vocab_input_size = len(eng_tokenizer.word_index) + 1
vocab_target_size = len(fra_tokenizer.word_index) + 1

In [26]:
embedding_dim = 128
units = 256
batch = 16
EPOCHES = 10

In [21]:
class BahdanauAttetion(Layer):
  def __init__(self,units):
    super().__init__()
    self.w1 =  Dense(units)
    self.w2 = Dense(units)
    self.V = Dense(1)

  def call(self,query,value):
    query_with_time_axis = tensorflow.expand_dims(query, 1)
    score = self.V(tensorflow.nn.tanh(self.w1(value) + self.w2(query_with_time_axis)))
    attention_weights = tensorflow.nn.softmax(score, axis=1)
    context_vector = tensorflow.reduce_sum(attention_weights * value, axis=1)
    return context_vector, tensorflow.squeeze(attention_weights, -1)


In [22]:
class Encoder(Model):
  def __init__(self, vocab_size, embedding_dim, enc_units):
    super(Encoder,self).__init__()
    self.enc_units = enc_units
    self.embedding = Embedding(vocab_size,embedding_dim)
    self.lstm = LSTM(enc_units, return_sequences=True, return_state=True)

  def call(self, x):
    x = self.embedding(x)
    output, state_h, state_c = self.lstm(x)
    return output, state_h, state_c


In [24]:
class Decoder(Model):
  def __init__(self, vocab_size, embedding_dim, dec_units):
    super(Decoder,self).__init__()
    self.dec_units = dec_units
    self.embedding = Embedding(vocab_size,embedding_dim)
    self.attention = BahdanauAttetion(dec_units)

    self.cell = tensorflow.keras.layers.LSTMCell(dec_units)
    self.dence  = Dense(vocab_size)

  def call(self, dec_input, enc_output, initial_state):
    batch_size = tensorflow.shape(dec_input)[0]
    dec_seq_len = tensorflow.shape(dec_input)[1]

    embedding = self.embedding(dec_input)
    outputs = []
    attention_weights_history = []

    states = initial_state

    for t in range(dec_seq_len):
      x_t = embedding[:, t, :]

      context_vector, attn_weight = self.attention(states[0], enc_output)

      x_and_context = tensorflow.concat([x_t, context_vector], axis=-1)
      output, states = self.cell(x_and_context, states)
      logits = self.dense(output)
      outputs.append(tensorflow.expand_dims(logits,1))
      attention_weights_history.append(tensorflow.expand_dims(attn_weight, 1))

    logits_seq = tensorflow.concat(outputs, axis=1)
    attention_history = tensorflow.concat(attention_weights_history, axis = 1)
    return logits_seq, states, attention_history



In [25]:
class Seq2Seq(Model):
  def __init__(self, encoder,decoder):
    super(Seq2Seq,self).__init__()
    self.encoder = encoder
    self.decoder = decoder

  def call(self, inputs):
    enc_in, dec_in = inputs

    enc_output,enc_h,enc_c = self.encoder(enc_in)
    logits, _, attn = self.decoder(dec_in, enc_output, (enc_h, enc_c))
    return logits


In [27]:
encoder = Encoder(vocab_input_size, embedding_dim, units)
decoder = Decoder(vocab_target_size, embedding_dim, units)
model = Seq2Seq(encoder,decoder)

In [28]:
loss_object = tensorflow.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [30]:
def loss_function(real, pred):
  mask = tensorflow.cast(tensorflow.not_equal(real, 0), dtype=tensorflow.float32)
  loss_ = loss_object(real, pred)
  loss_ *= mask
  return tensorflow.reduce_sum(loss_) / (tensorflow.reduce_sum(mask) + le-9)

In [31]:
model.compile(optimizer='adam',loss=loss_function, metrics=['accuracy'])

In [32]:
model.summary()

Model: "seq2_seq"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder (Encoder)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Decoder)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)